---
jupyter: python3
---

In [ ]:
# DeepXDE Basics

##### (c) J. Rafael Rodríguez Galván 2024

In [ ]:
#!pip install deepxde

In [ ]:
import deepxde as dde
dde.backend.get_available_backend()

## A simple ode
Example from [DeepXDE web page](https://deepxde.readthedocs.io/en/latest/demos/pinn_forward/ode.system.html)

\begin{align}
y_1' &= y_2,\\
y_2' &= -y_1,
\end{align}
with initial condition $(y_1(0), y_2(0))=(0,1)$.
Exact solution: $(y_1(x), y_2(x))=(\sin(x), \cos(x))$.

In [ ]:
"""Backend supported: tensorflow.compat.v1, tensorflow, pytorch, jax, paddle"""
import deepxde as dde
import numpy as np


def ode_system(x, y):
    """ODE system.
    dy1/dx = y2
    dy2/dx = -y1
    """
    # Most backends (see DeepXDE for more detais)
    y1, y2 = y[:, 0:1], y[:, 1:]
    dy1_x = dde.grad.jacobian(y, x, i=0)
    dy2_x = dde.grad.jacobian(y, x, i=1)
    return [dy1_x - y2, dy2_x + y1]


def boundary(_, on_initial):
    return on_initial


def func(x):
    """
    y1 = sin(x)
    y2 = cos(x)
    """
    return np.hstack((np.sin(x), np.cos(x)))


geom = dde.geometry.TimeDomain(0, 10)
ic1 = dde.icbc.IC(geom, lambda x: 0, boundary, component=0)
ic2 = dde.icbc.IC(geom, lambda x: 1, boundary, component=1)
data = dde.data.PDE(geom, ode_system, [ic1, ic2], 35, 2, solution=func, num_test=100)

layer_size = [1] + [50] * 3 + [2]
activation = "tanh"
initializer = "Glorot uniform"
net = dde.nn.FNN(layer_size, activation, initializer)

model = dde.Model(data, net)
model.compile("adam", lr=0.001, metrics=["l2 relative error"])
losshistory, train_state = model.train(iterations=20000)

dde.saveplot(losshistory, train_state, issave=True, isplot=True)